In [1]:
#Dependencies

import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Extract CSVs into Dataframes

features_file = "Resources/SpotifyFeatures.csv"
features_df = pd.read_csv(features_file)
features_df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [3]:
top2018_file = "Resources/top2018.csv"
top2018_df = pd.read_csv(top2018_file)
top2018_df.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6DCZcSspjsKoFjzjrWoCd,God's Plan,Drake,0.754,0.449,7.0,-9.211,1.0,0.1090,0.0332,0.000083,0.552,0.357,77.169,198973.0,4.0
1,3ee8Jmje8o58CHK66QrVC,SAD!,XXXTENTACION,0.740,0.613,8.0,-4.880,1.0,0.1450,0.2580,0.003720,0.123,0.473,75.023,166606.0,4.0
2,0e7ipj03S05BNilyu5bRz,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5.0,-6.090,0.0,0.0898,0.1170,0.000066,0.131,0.140,159.847,218147.0,4.0
3,3swc6WTsr7rl9DqQKQA55,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8.0,-8.011,1.0,0.1170,0.5800,0.000000,0.112,0.439,140.124,221440.0,4.0
4,2G7V7zsVDxg1yRsu7Ew9R,In My Feelings,Drake,0.835,0.626,1.0,-5.833,1.0,0.1250,0.0589,0.000060,0.396,0.350,91.030,217925.0,4.0


In [14]:
# Transform features DataFrame 
#key in the top2018 data set is recorded different then the features data set. Going to remove key from the columns
features_cols = ["track_id","track_name", "artist_name",  "popularity","danceability", "valence"]
features_transformed= features_df[features_cols].copy()

# Rename the column headers and change the order
features_transformed = features_transformed.rename(columns={ "track_id": "id",
                                                            "track_name": "song",
                                                            "artist_name": "artist",
                                                            "popularity":"popularity",
                                                            "danceability":"danceability",
                                                            "valence":"valence"     
                                                          })
                                                            
# Clean the data by dropping duplicates and set index
features_transformed.drop_duplicates("id", inplace=True)
features_transformed.set_index("id", inplace=True)

features_transformed.head()

,song,artist,popularity,danceability,valence
id,,,,,
0BRjO6ga9RKCKjfDqeFgWV,C'est beau de faire un Show,Henri Salvador,0,0.389,0.814
0BjC1NfoEOOusryehmNudP,Perdu d'avance (par Gad Elmaleh),Martin & les fées,1,0.590,0.816
0CoSDzoNIKCRs124s9uTVy,Don't Let Me Be Lonely Tonight,Joseph Williams,3,0.663,0.368
0Gc6TVm52BwZD07Ki6tIvf,Dis-moi Monsieur Gordon Cooper,Henri Salvador,0,0.240,0.227
0IuslXpMROHdEPvSl1fTQK,Ouverture,Fabien Nataf,4,0.331,0.390


In [15]:
###arranging the data set with the best popularity 
popfeatures = features_transformed.sort_values('popularity', ascending=[False])
popfeatures.head(20)

,song,artist,popularity,danceability,valence
id,,,,,
14msK75pk3pA33pzPVNtBF,7 rings,Ariana Grande,100,0.725,0.319
4kV4N9D1iKVxx1KLvtTpjS,"break up with your girlfriend, i'm bored",Ariana Grande,99,0.726,0.335
6MWtB6iiXyIwun0YzU6DFP,Wow.,Post Malone,99,0.833,0.385
5w9c2J52mkdntKOmRLeM2m,Con Calma,Daddy Yankee,98,0.737,0.656
3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,97,0.760,0.913
5p7ujcrUXASCNwRaWNHR1C,Without Me,Halsey,97,0.752,0.533
2dpaYNEQHiRxtZbfNsse99,Happier,Marshmello,97,0.687,0.671
25sgk305KZfyuqVBQIahim,Sweet but Psycho,Ava Max,97,0.719,0.628
6Qs4SXO9dwPj5GKvVOv8Ki,Dancing With A Stranger (with Normani),Sam Smith,97,0.741,0.347


In [7]:
# Transform top2018 DataFrame
top2018_cols = ["id", "name", "artists","danceability", "valence"]
top2018_transformed= top2018_df[top2018_cols].copy()

# Rename the column headers
top2018_transformed = top2018_transformed.rename(columns={"id": "id", 
                                                          "artists": "artist", 
                                                          "name": "song",
                                                         "danceability":"danceability",
                                                            "valence":"valence" })

# Set index
top2018_transformed.set_index("id", inplace=True)

top2018_transformed.head()

,song,artist,danceability,valence
id,,,,
6DCZcSspjsKoFjzjrWoCd,God's Plan,Drake,0.754,0.357
3ee8Jmje8o58CHK66QrVC,SAD!,XXXTENTACION,0.740,0.473
0e7ipj03S05BNilyu5bRz,rockstar (feat. 21 Savage),Post Malone,0.587,0.140
3swc6WTsr7rl9DqQKQA55,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.439
2G7V7zsVDxg1yRsu7Ew9R,In My Feelings,Drake,0.835,0.350


In [8]:
#arranging the data set with the best dancibility 
dancetop2018 = top2018_transformed.sort_values('danceability', ascending=[False])
dancetop2018.head(20)

,song,artist,danceability,valence
id,,,,
6vN77lE9LK6HP2DewaN6H,Yes Indeed,Lil Baby,0.964,0.562
2E124GmJRnBJuXbTb4cPU,FEFE (feat. Nicki Minaj & Murda Beatz),6ix9ine,0.931,0.376
4qKcDkK6siZ7Jp1Jb4m0a,Look Alive (feat. Drake),BlocBoy JB,0.922,0.595
0JP9xo3adEtGSdUEISisz,Moonlight,XXXTENTACION,0.921,0.711
0TlLq3lA83rQOYtrqBqSc,Nonstop,Drake,0.912,0.422
3xcCix7Jv1Rp90YVmgo35,Bella,Wolfine,0.909,0.844
6n4U3TlzUGhdSFbUUhTvL,Walk It Talk It,Migos,0.909,0.406
7KXjTSCq5nL1LoYtL7XAw,HUMBLE.,Kendrick Lamar,0.908,0.421
3V8UKqhEK5zBkBb6d6ub8,Te Bot? - Remix,Nio Garcia,0.903,0.442


In [ ]:
#key in the top2018 data set is recorded different then the features data set. Going to remove key from the columns

In [ ]:
# Create database connection (Be Sure to add your password and if you receive
# a localhost error, try 5432 instead of 5433!) - Addie please run this 
#connection_string = "postgres:Bearbee1!@localhost:5432/spotify_db"
#engine = create_engine(f'postgresql://{connection_string}')

In [16]:
# Create database connection (Be Sure to add your password and if you receive
# a localhost error, try 5432 instead of 5433!)
connection_string = "postgres:brandup2.0@localhost:5432/spotify_db"
engine = create_engine(f'postgresql://{connection_string}')

In [26]:
# Confirm tables 
engine.table_names()

['features', 'top2018']

In [35]:
#Load DataFrames into database 
popfeatures.to_sql(name='features', con=engine, if_exists='append', index=True)

In [39]:
#Load DataFrames into database
dancetop2018.to_sql(name='top2018', con=engine, if_exists='append', index=True)

In [40]:
pd.read_sql_query('select * from top2018', con=engine).head()

,id,song,artist,danceability,valence
0,6vN77lE9LK6HP2DewaN6H,Yes Indeed,Lil Baby,1.0,0.6
1,2E124GmJRnBJuXbTb4cPU,FEFE (feat. Nicki Minaj & Murda Beatz),6ix9ine,0.9,0.4
2,4qKcDkK6siZ7Jp1Jb4m0a,Look Alive (feat. Drake),BlocBoy JB,0.9,0.6
3,0JP9xo3adEtGSdUEISisz,Moonlight,XXXTENTACION,0.9,0.7
4,0TlLq3lA83rQOYtrqBqSc,Nonstop,Drake,0.9,0.4


In [38]:
pd.read_sql_query('select * from features', con=engine).head()

,id,song,artist,popularity,danceability,valence
0,14msK75pk3pA33pzPVNtBF,7 rings,Ariana Grande,100,0.7,0.3
1,4kV4N9D1iKVxx1KLvtTpjS,"break up with your girlfriend, i'm bored",Ariana Grande,99,0.7,0.3
2,6MWtB6iiXyIwun0YzU6DFP,Wow.,Post Malone,99,0.8,0.4
3,5w9c2J52mkdntKOmRLeM2m,Con Calma,Daddy Yankee,98,0.7,0.7
4,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,97,0.8,0.9
